In [3]:
%cd ~/Dropbox/CanvasTools

import environment
import os
import sys
import configparser

import requests

# Load api token and section numbers
print("Reading credentials from %s" % environment.CREDENTIALS_FILE)
config = configparser.ConfigParser()
config.read( environment.CREDENTIALS_FILE)

TOKEN = config['credentials'].get('TOKEN')
SECTION_930 = config['sections'].getint('930')
SECTION_1100 = config['sections'].getint('1100')
SECTION_1230 = config['sections'].getint('1230')
URL_BASE = config['url'].get('BASE')

/Users/adam/Dropbox/CanvasTools
Reading credentials from /Users/adam/Dropbox/CanvasTools/private/canvas-credentials.ini


In [4]:
from CanvasTools.UrlTools import make_url
from CanvasTools.RequestTools import send_get_request

# Get list of quizzes

 GET /api/v1/courses/:course_id/quizzes 

In [ ]:
def get_all_quizzes(course_id):
    url = make_url(SECTION_930, 'quizzes')

In [ ]:
# Get quiz list
def get_all_course_quizzes(course_id):
    """Returns a list of all the assignments for the course
    Uses api: GET /api/v1/courses/:course_id/quizzes
    """
    url = make_url(course_id, 'quizzes')
    return send_get_request(url)


def get_quiz(course_id, quiz_id):
    """Retrieves the specified quiz and returns it as a dictionary"""
    url = make_url(course_id, 'quizzes')
    url = "%s/%s" % (url, quiz_id)
    print(url)
#     return send_get_request(url)

In [ ]:
get_all_course_quizzes(SECTION_930)

In [ ]:
get_quiz(SECTION_930, 100843)

In [ ]:
# get all quiz submissions
# /api/v1/courses/:course_id/quizzes/:quiz_id/submissions 

In [ ]:
def get_quiz_submissions(course_id, quiz_id):
    url = make_url(course_id, 'quizzes')
    url = "%s/%s/submissions" % (url, quiz_id)
    print(url)
    return send_get_request(url)

In [ ]:
get_quiz_submissions(SECTION_930, 100843)

In [ ]:
 GET /api/v1/quiz_submissions/:quiz_submission_id/questions 

In [ ]:
def get_answers(submission_id):
    url = "https://canvas.csun.edu/api/v1/quiz_submissions/%s/questions" % (submission_id)
    return send_get_request(url)


In [ ]:
get_answers(9789591)

In [ ]:
submission_id = 9789593
# url = "https://canvas.csun.edu/api/v1/quiz_submissions/%s/questions" % (submission_id)
url = "https://canvas.csun.edu/api/v1/courses/41179/quiz_submissions/%s/questions"  % (submission_id)
# url = 'https://canvas.csun.edu/courses/41179/quizzes/100843/history?quiz_submission_id=1158384&version=1'
head = {'Authorization': 'Bearer {}'.format(environment.TOKEN)}
response = requests.get(url, headers=head)


In [ ]:
response.status_code

In [ ]:
response.text

# reports

The URL to use is /api/v1/courses/{CourseId}/quizzes/{QuizId}/reports. Add a form parameter to the header : quiz_report[report_type]=student_analysis. You will get back a Json object and two of its members are a filename and report id. Use the report id to go to /api/v1/courses/{CourseId}/quizzes/{QuizId}/reports/{ReportId}. This will then return the downloaded csv file.

 

In [6]:
course_id = SECTION_930
quiz_id = 100843
report_id = 40084

url = make_url(course_id, 'quizzes')
head = {'Authorization': 'Bearer {}'.format(environment.TOKEN)}


In [ ]:
# Create the student report
url = "%s/%s/reports" % (url, quiz_id)
dt = {'quiz_report': {'report_type': 'student_analysis'}, 'include': 'file'}
print(url)

response1 = requests.post(url, headers=head, json=dt)
r1 = response1.json()

# report_id = 

In [7]:
# retrieve the report file information
url = "%s/%s" % (url, report_id)
dt = { 'include': 'file'}
print(url)
response2 = requests.get(url, headers=head, json=dt)
r2 = response2.json()

# Link to the resource 
file_url = r2['file']['url']
file_name =  r2['file']['display_name']
print(file_url)
print(file_name)

https://canvas.csun.edu/api/v1/courses/41179/quizzes/100843/reports/40084
https://canvas.csun.edu/files/4676929/download?download_frd=1&verifier=z1ERTI1raBBIhIzinTkVp9mpPQx2zihJDXb3cY9S
Persuasive advertising, autonomy, and the creation of desire Quiz Student Analysis Report.csv


In [9]:
# download file
file_response = requests.get(file_url, headers=head)
# response.json()

# write to temp
destination = "%s/%s" % (environment.TEMP_DATA_PATH, file_name)
open(destination, 'wb').write(file_response.content)

77268

In [ ]:
response.content

# submissions

In [ ]:
submission_id = 1164462
course_id = SECTION_930
quiz_id = 100843

url = make_url(course_id, 'quizzes')
url = "%s/%s/submissions/%s" % (url, quiz_id, submission_id )
print(url)

head = {'Authorization': 'Bearer {}'.format(environment.TOKEN),
       'include': 'quiz_question'
       }
response = requests.get(url, headers=head)


In [ ]:
response.json()

In [ ]:
# make reports
url = "https://canvas.csun.edu/api/v1/courses/41179/quizzes/100843/reports/40084"
head = {'Authorization': 'Bearer {}'.format(environment.TOKEN)
       }
response = requests.get(url, headers=head)


In [ ]:
response

In [ ]:
response.text